In [1]:
#| default_exp smoothing

In [2]:
#| export

from blender_tissue_cartography import io as tcio
from blender_tissue_cartography import registration as tcreg

import numpy as np
from copy import deepcopy
import warnings
import igl

from scipy import sparse

In [3]:
from blender_tissue_cartography import interpolation as tcinterp
from skimage import transform
import matplotlib.pyplot as plt

In [4]:
from tqdm.notebook import tqdm
from importlib import reload

In [5]:
np.set_printoptions(suppress=True)

## Mesh smoothing

Here, we implement several mesh smoothing methods using `libigl`.

In [5]:
# load a test mesh
mesh_registered = tcio.ObjMesh.read_obj(f"wrapping_example/Drosophila_reference_registered.obj")

## Procedural wrapping

We can also automatically wrap using blender's python API. Let's see how it works. 
- Annoyingly, blender scripts cannot be directly run from python. It must be done from within the blender script window. The goal here would be to write a script that can be either easily run in blender, or run from the command line (also possible via python's `shutil`).
- `libigl`'s `point_mesh_squared_distance`

### Smoothing in `libigl`

We implement Laplacian and Taubin smoothing for vertex positions using `libigl`'s Laplacian operator.

In [6]:
#| export

def get_uniform_laplacian(tris, normalize=True):
    """
    Get uniform Laplacian (purely connectivity based) as sparse matrix.
    
    If normalize, the diagonal = -1. Else, diagonal equals number of neighbors.
    """
    a = igl.adjacency_matrix(tris)
    a_sum = np.squeeze(np.asarray(a.sum(axis=1)))
    a_diag = sparse.diags(a_sum)
    if normalize:
        return sparse.diags(1/a_sum)@(a - a_diag)
    return (a - a_diag)

In [7]:
#| export

def smooth_laplacian(mesh: tcio.ObjMesh, lamb=0.5, n_iter=10, method="explicit", boundary="fixed") -> tcio.ObjMesh:
    """
    Smooth mesh vertex positions using Laplacian filter.
    
    Assumes mesh is triangular.
    
    Parameters
    ----------
    mesh : ObjMesh
        Initial mesh.
    lamb : float, default 0.5
        Filter strength. Higher = more smoothing.
    n_iter : int
        Filter iterations
    method : str, default "explicit"
        Can use explicit (fast, simple) or implicit (slow, more accurate) method.
    boundary : str, "fixed" or "free"
        Whether to allow mesh boundary to move

    Returns
    -------
    mesh_smoothed : ObjMesh
        Smoothed mesh.

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular - result may be incorrect", RuntimeWarning)
    v_smoothed = np.copy(mesh.vertices)
    f = mesh.tris
    boundary_vertices = igl.boundary_facets(f)[:, 0]

    if method == "implicit":
        laplacian = igl.cotmatrix(v_smoothed, f)
        for _ in range(n_iter):
            mass = igl.massmatrix(v_smoothed, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
            v_smoothed = sparse.linalg.spsolve(mass - lamb * laplacian, mass.dot(v_smoothed))
            if boundary == "fixed":
                v_smoothed[boundary_vertices] = mesh.vertices[boundary_vertices]
    elif method == "explicit":
        laplacian_uniform = get_uniform_laplacian(f)
        for _ in range(n_iter):
            v_smoothed += lamb*laplacian_uniform.dot(v_smoothed)
            if boundary == "fixed":
                v_smoothed[boundary_vertices] = mesh.vertices[boundary_vertices]
    mesh_smoothed = tcio.ObjMesh(v_smoothed, mesh.faces, texture_vertices=mesh.texture_vertices,
                                 normals=None, name=mesh.name)
    mesh_smoothed.set_normals()
    return mesh_smoothed

In [9]:
mesh_texture_smoothed = smooth_laplacian_texture(mesh_registered, lamb=0.5, n_iter=100, boundary="fixed")

In [10]:
boundary_vertices = igl.boundary_facets(mesh_registered.texture_tris)[:, 0]

In [11]:
np.allclose(mesh_texture_smoothed.texture_vertices[boundary_vertices],
            mesh_registered.texture_vertices[boundary_vertices])

True

In [12]:
mesh_smoothed = smooth_laplacian(mesh_registered, lamb=0.5, n_iter=10, method="explicit")

In [13]:
mesh_smoothed.write_obj("wrapping_example/Drosophila_reference_smoothed_uniform_igl.obj")

In [14]:
#| export

def smooth_taubin(mesh: tcio.ObjMesh, lamb=0.5, nu=0.53, n_iter=10,) -> tcio.ObjMesh:
    """
    Smooth using Taubin filter (like Laplacian, but avoids shrinkage).
    
    Assumes mesh is triangular. See   "Improved Laplacian Smoothing of Noisy Surface Meshes"
    J. Vollmer, R. Mencl, and H. Muller.
    
    Parameters
    ----------
    mesh : ObjMesh
        Initial mesh.
    lamb : float, default 0.5
        Filter strength. Higher = more smoothing.
    nu : float, default 0.53
        Counteract shrinkage. Higher = more dilation.
    n_iter : int
        Filter iterations

    Returns
    -------
    mesh_smoothed : ObjMesh
        Smoothed mesh.

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular - result may be incorrect", RuntimeWarning)
    v_smoothed = np.copy(mesh.vertices)
    laplacian_uniform = get_uniform_laplacian(mesh.tris)
    for _ in range(n_iter):
        v_smoothed += lamb*laplacian_uniform.dot(v_smoothed)
        v_smoothed -= nu*laplacian_uniform.dot(v_smoothed)
    mesh_smoothed = tcio.ObjMesh(v_smoothed, mesh.faces, texture_vertices=mesh.texture_vertices,
                                 normals=None, name=mesh.name)
    mesh_smoothed.set_normals()
    return mesh_smoothed

In [15]:
mesh_smoothed_taubin = smooth_taubin(mesh_registered, lamb=0.5, nu=0.53, n_iter=10)

In [16]:
mesh_smoothed_taubin.write_obj("wrapping_example/Drosophila_reference_smoothed_taubin_igl.obj")

## Texture smoothing

Sometimes, UV maps can become very deformed, or even display self-intersection. Smoothing can fix this.

In [8]:
#| export

def smooth_laplacian_texture(mesh: tcio.ObjMesh, lamb=0.5, n_iter=10, boundary="fixed") -> tcio.ObjMesh:
    """
    Smooth mesh texture positions using Laplacian filter.
    
    This function is very helpful to fix UV maps with flipped triangles, as detected by
    igl.flipped_triangles. Assumes mesh is triangular.
    
    Parameters
    ----------
    mesh : ObjMesh
        Initial mesh.
    lamb : float, default 0.5
        Filter strength. Higher = more smoothing.
    n_iter : int
        Filter iterations
    boundary : str, "fixed" or "free"
         Whether to allow UV "island" boundary to move

    Returns
    -------
    mesh_smoothed : ObjMesh
        Smoothed mesh.

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular - result may be incorrect", RuntimeWarning)
    v_smoothed = np.copy(mesh.texture_vertices)
    f = mesh.texture_tris
    laplacian_uniform = get_uniform_laplacian(f)
    boundary_vertices = igl.boundary_facets(f)[:, 0]

    for _ in range(n_iter):
        v_smoothed += lamb*laplacian_uniform.dot(v_smoothed)
        if boundary == "fixed":
            v_smoothed[boundary_vertices] = mesh.texture_vertices[boundary_vertices]
    mesh_smoothed = tcio.ObjMesh(mesh.vertices, mesh.faces, texture_vertices=v_smoothed,
                                 normals=mesh.normals, name=mesh.name)
    return mesh_smoothed

For strongly degraded UV maps with many self-intersection (flipped triangles), a more radical appoach can help - fix only the UV island boundaries, and recompute all UV vertex positions using a harmonic map. This fixes the self-intersection issues. Such UV maps can for example result from a spherical or cylindrical projection of a very deformed object.

### On-surface smoothing

Smooth, but project at each step so the mesh vertices are back on the surface. This is very useful to smooth out surface-surface maps.

In [33]:
#| export

def smooth_laplacian_on_surface(mesh: tcio.ObjMesh, n_iter=10, lamb=0.5, n_iter_laplace=10,
                                boundary="fixed") -> tcio.ObjMesh:
    """
    Smooth mesh vertex positions using Laplacian filter and project vertices back to original surface.
    
    Alternates between Laplacian smoothing and projecting back to original surface. Uses
    explicit method for Laplactian smoothing
    
    Parameters
    ----------
    mesh : ObjMesh
        Initial mesh.
    n_iter : int
        Number of iterations at each step
    lamb : float, default 0.5
        Filter strength. Higher = more smoothing.
    n_iter_laplace : int
        Laplace filter iterations. If reprojection messes upt your mesh, decrease this number.
    boundary : str, "fixed" or "free"
        Whether to allow mesh boundary to move

    Returns
    -------
    mesh_smoothed : ObjMesh
        Smoothed mesh.

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular - result may be incorrect", RuntimeWarning)
    v_reference = np.copy(mesh.vertices)
    v_smoothed = np.copy(mesh.vertices)
    f = mesh.tris
    boundary_vertices = igl.boundary_facets(f)[:, 0]
    laplacian_uniform = get_uniform_laplacian(f)

    for _ in range(n_iter):
        # smooth
        for _ in range(n_iter):
            v_smoothed += lamb*laplacian_uniform.dot(v_smoothed)
            if boundary == "fixed":
                v_smoothed[boundary_vertices] = mesh.vertices[boundary_vertices]
        # project 
        _, _, v_smoothed = igl.point_mesh_squared_distance(v_smoothed, v_reference, f)
        v_reference = np.copy(v_smoothed)
    mesh_smoothed = tcio.ObjMesh(v_smoothed, mesh.faces, texture_vertices=mesh.texture_vertices,
                                 normals=mesh.normals, name=mesh.name)
    mesh_smoothed.set_normals()
    return mesh_smoothed

In [34]:
mesh_test = tcio.ObjMesh.read_obj(f"movie_example/meshes_wrapped_reverse/mesh_01_wrapped_reverse.obj")

In [35]:
mesh_smoothed = smooth_laplacian_on_surface(mesh_test, n_iter=20, lamb=0.5, n_iter_laplace=5)

In [36]:
mesh_smoothed.write_obj(f"movie_example/on_surface_smooth.obj")